# Imports 

to do:  
    - make map for suspected crimes that can work for all data 
    - need to use NLP to parse crimes

In [1]:
# importing padas, numpy, matplotlib, statsmodels, statsmodels tsa, 
# datetime, and auto arima 

import pandas as pd
from datetime import time

# Functions 

In [2]:
def format_csv_dfs(dataframe):
    
    """
    DOC STRING:
    inputs a dataframe with all columns
    outputs a dataframe with only the time, race, precinct information, 
    and suspected crime
    """
    
    columns_to_keep = ['DATE_TIME', 'SUSPECT_ARRESTED_FLAG', 'SUSPECT_RACE_DESCRIPTION',
                       'STOP_LOCATION_PRECINCT', 'SUSPECTED_CRIME_DESCRIPTION',
                      'Stop Frisk Time', 'race', 'datestop', 'timestop', 'arstmade',
                      'pct', 'crimsusp']
    
    dataframe = dataframe.drop(columns=[col for col in dataframe.columns if col not in columns_to_keep])
    
    return dataframe

In [3]:
# a small function to remove the seconds
def format_twenty_and_nineteen(date):
    date = date[:-3]
    return date

In [4]:
# a small function to format 2018's datetimes
def format_eighteen(date):         
    date = date[0:11] + date[20:-3]
    return date

In [5]:
# a small function to format 2017's datetimes
def format_seventeen(date):
    date = date[:16]
    if 'nan' in date:
        date = date[:10] + ' 00:00:00'
    return date

In [6]:
# a small function to reformat precincts and remove nonsensical precincts,
# makes the rows with empty precinct into 0 so we don't lose data

def reformat_precinct(x):
    if '.0' in x:
        x = x.split('.0')
        x = x[0]
    return x

In [7]:
def reformat_date_time(df):
    time_replaced = []
    date_replaced = []
    for x in df['DATE_TIME']:
        x = x.split(' ')
        date = x[0]
        time = x[1]
        if time == '' or len == 0:
            time_replace = "00:00"
        if len(time) == 1: 
            time_replace = "0" + time + ":00"
        #example "3" turns into "0300"
        if len(time) == 2: 
            time_replace = "00:" + time
        #example "30" turns into "0030"
        if len(time) == 3: 
            time_replace = "0" + time[0] + ":" + time[1::]
        #exapmle "130" turns into 0130
        if len(time) == 4: 
            time_replace = time[0:2] + ":" + time[2::]
        #example "1130" turns into 11:30
        time_replaced.append(time_replace)

        if len(date) == 7:
            date_replace = "0" + date[0] + '/' + date[1:3] + '/' + date [3::]
            #example "2072016" into "02/07/2016"
        if len(date) == 8:
            date_replace = date[0:2] + "/" + date [2:4] + '/' + date [4::]
            #exapmle '10082016' into '10/08/2016'
        date_replaced.append(date_replace)

    date_time = list(zip(date_replaced, time_replaced))
    date_time_formatted = []
    for x in date_time:
        x = x[0] + ' ' + x[1]
        date_time_formatted.append(x)
    return date_time_formatted

# Loading in Data from the Last 10 Years

In [8]:
#df_twentyone = pd.read_excel('SF_2021.xlsx', parse_dates=[['STOP_FRISK_DATE',
 #                                                      'STOP_FRISK_TIME']])
#df_twentyone.to_csv("SF_2021.csv.gz", compression = 'gzip', index = False)

In [9]:
###df_twentyone = pd.read_excel('SF_2021.xlsx', parse_dates=[['STOP_FRISK_DATE',
###                                                       'STOP_FRISK_TIME']])
###df_twentyone.to_csv("SF_2021.csv.gz", compression = 'gzip', index = False)


# loading in 2020 data, while parsing the dates, shrinking the data and 
# reloading the data, and sending the datetime column through all the reformating
df_twentyone = pd.read_csv("raw_data/SF_2021.csv.gz")
df_twentyone.head()

df_twentyone.rename({'STOP_FRISK_DATE_STOP_FRISK_TIME' : 'DATE_TIME'}, axis = 1, inplace = True)


df_twentyone = format_csv_dfs(df_twentyone)
df_twentyone['DATE_TIME'] = df_twentyone['DATE_TIME'].map(format_twenty_and_nineteen)#.map(to_month_day_year).map(time_logic_check)

/Users/b0ihazard/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
df_twentyone.head()

,DATE_TIME,SUSPECTED_CRIME_DESCRIPTION,SUSPECT_ARRESTED_FLAG,SUSPECT_RACE_DESCRIPTION,STOP_LOCATION_PRECINCT
0,2021-01-01 01:50,CPW,Y,BLACK,42
1,2021-01-01 10:55,ROBBERY,Y,BLACK,105
2,2021-01-01 10:10,OTHER,Y,ASIAN / PACIFIC ISLANDER,106
3,2021-01-01 20:17,ASSAULT,N,BLACK,73
4,2021-01-01 19:59,RAPE,N,BLACK,26


In [11]:
###df_twenty = pd.read_excel('SF_2020.xlsx', parse_dates=[['STOP_FRISK_DATE',
#                                                        'STOP_FRISK_TIME']])
###df_twenty.to_csv("SF_2020.csv.gz", compression = 'gzip', index = False)


# loading in 2020 data, while parsing the dates, shrinking the data and 
# reloading the data, and sending the datetime column through all the reformating
df_twenty = pd.read_csv("raw_data/SF_2020.csv.gz")

df_twenty.rename({'STOP_FRISK_DATE_STOP_FRISK_TIME' : 'DATE_TIME'}, axis = 1, inplace = True)

df_twenty.columns

df_twenty = format_csv_dfs(df_twenty)
df_twenty['DATE_TIME'] = df_twenty['DATE_TIME'].map(format_twenty_and_nineteen)#.map(to_month_day_year).map(time_logic_check)

In [12]:
df_twenty.head()
# inspecting the data

,DATE_TIME,SUSPECTED_CRIME_DESCRIPTION,SUSPECT_ARRESTED_FLAG,SUSPECT_RACE_DESCRIPTION,STOP_LOCATION_PRECINCT
0,2020-01-01 01:12,CPW,Y,BLACK,19
1,2020-01-01 01:11,CPW,N,BLACK,19
2,2020-01-01 01:11,CPW,N,WHITE HISPANIC,19
3,2020-01-01 10:18,MENACING,Y,BLACK,75
4,2020-01-01 08:45,BURGLARY,N,WHITE,78


In [13]:
#df_nineteen = pd.read_excel('SF_2019.xlsx', parse_dates=[['STOP_FRISK_DATE',
#                                                        'STOP_FRISK_TIME']])
#df_nineteen.to_csv("SF_2019.csv.gz", compression = 'gzip', index = False)

# loading in 2019 data, while parsing the dates, shrinking the data and 
# reloading the data, and sending the datetime column through all the reformating

df_nineteen = pd.read_csv("raw_data/SF_2019.csv.gz")

df_nineteen.rename({'STOP_FRISK_DATE_STOP_FRISK_TIME' : 'DATE_TIME'}, axis = 1, inplace = True)

df_nineteen = format_csv_dfs(df_nineteen)
df_nineteen['DATE_TIME'] = df_nineteen['DATE_TIME'].map(format_twenty_and_nineteen)#.map(to_month_day_year).map(time_logic_check)

In [14]:
df_nineteen.head()
# inspecting the data

,DATE_TIME,SUSPECTED_CRIME_DESCRIPTION,SUSPECT_ARRESTED_FLAG,SUSPECT_RACE_DESCRIPTION,STOP_LOCATION_PRECINCT
0,2019-01-02 14:30,PETIT LARCENY,N,BLACK,1
1,2019-01-08 02:30,GRAND LARCENY,Y,BLACK,1
2,2019-01-12 16:54,PETIT LARCENY,Y,WHITE,1
3,2019-01-14 21:21,ROBBERY,N,BLACK,1
4,2019-01-15 18:50,ASSAULT,Y,BLACK,1


In [15]:
#df_eighteen = pd.read_excel('SF_2018.xlsx', parse_dates=[['STOP_FRISK_DATE',
#                                                        'Stop Frisk Time']])

#df_eighteen.to_csv("SF_2018.csv.gz", compression = 'gzip', index = False)

# loading in 2018 data, while parsing the dates, shrinking the data and 
# reloading the data, and sending the datetime column through all the reformating
df_eighteen = pd.read_csv("raw_data/SF_2018.csv.gz")

df_eighteen.rename({'STOP_FRISK_DATE_Stop Frisk Time' : 'DATE_TIME'}, axis = 1, inplace = True)

df_eighteen = format_csv_dfs(df_eighteen)
df_eighteen['DATE_TIME'] = df_eighteen['DATE_TIME'].map(format_eighteen)#.map(to_month_day_year).map(time_logic_check)

In [16]:
df_eighteen.head()
#inspecting the data 

,DATE_TIME,SUSPECTED_CRIME_DESCRIPTION,SUSPECT_ARRESTED_FLAG,SUSPECT_RACE_DESCRIPTION,STOP_LOCATION_PRECINCT
0,2018-01-01 19:04,MENACING,N,WHITE,1
1,2018-01-01 23:00,CPW,N,BLACK HISPANIC,34
2,2018-01-01 23:55,GRAND LARCENY,N,WHITE HISPANIC,43
3,2018-01-01 03:23,ROBBERY,Y,AMERICAN INDIAN/ALASKAN NATIVE,63
4,2018-01-01 03:23,ROBBERY,Y,WHITE,63


In [17]:
#df_seventeen = pd.read_excel('SF_2017.xlsx', parse_dates = [['STOP_FRISK_DATE',
#                                                        'STOP_FRISK_TIME']])

#df_seventeen.to_csv("SF_2017.csv.gz", compression = 'gzip', index = False)

# loading in 2017 data, while parsing the dates, shrinking the data and 
# reloading the data, and sending the datetime column through all the reformating

df_seventeen = pd.read_csv("raw_data/SF_2017.csv.gz")

df_seventeen.rename({'STOP_FRISK_DATE_STOP_FRISK_TIME' : 'DATE_TIME'}, axis = 1, inplace = True)

df_seventeen = format_csv_dfs(df_seventeen)
df_seventeen['DATE_TIME'] = df_seventeen['DATE_TIME'].map(format_seventeen)#.map(to_month_day_year).map(time_logic_check)

In [18]:
df_seventeen.head()

,DATE_TIME,SUSPECTED_CRIME_DESCRIPTION,SUSPECT_ARRESTED_FLAG,SUSPECT_RACE_DESCRIPTION,STOP_LOCATION_PRECINCT
0,2017-01-16 14:25,TERRORISM,N,(null),1.0
1,2017-01-16 14:25,TERRORISM,N,(null),1.0
2,2017-02-08 11:09,OTHER,N,WHITE,1.0
3,2017-02-20 11:34,GRAND LARCENY AUTO,N,BLACK HISPANIC,1.0
4,2017-02-21 13:20,BURGLARY,N,BLACK,1.0


## first DF 2021-2017

In [19]:
# combining df20-df17
dfs_twentyone_to_seventeen = [df_twentyone, df_twenty, df_nineteen, df_eighteen, df_seventeen]

df_1 = pd.concat(dfs_twentyone_to_seventeen)
df_1.rename(columns = {'STOP_LOCATION_PRECINCT' : 'PRECINCT'}, inplace = True)
df_1.head()

,DATE_TIME,SUSPECTED_CRIME_DESCRIPTION,SUSPECT_ARRESTED_FLAG,SUSPECT_RACE_DESCRIPTION,PRECINCT
0,2021-01-01 01:50,CPW,Y,BLACK,42.0
1,2021-01-01 10:55,ROBBERY,Y,BLACK,105.0
2,2021-01-01 10:10,OTHER,Y,ASIAN / PACIFIC ISLANDER,106.0
3,2021-01-01 20:17,ASSAULT,N,BLACK,73.0
4,2021-01-01 19:59,RAPE,N,BLACK,26.0


In [20]:
for x in df_1['DATE_TIME']:
    print(x)
# inspecting the date time column to make sure all the reformatting took

2021-01-01 01:50
2021-01-01 10:55
2021-01-01 10:10
2021-01-01 20:17
2021-01-01 19:59
2021-01-01 21:29
2021-01-01 16:33
2021-01-02 20:48
2021-01-02 19:50
2021-01-02 22:34
2021-01-02 22:12
2021-01-02 19:15
2021-01-01 22:20
2021-01-01 16:10
2021-01-02 14:56
2021-01-04 04:24
2021-01-03 23:41
2021-01-04 09:06
2021-01-04 09:09
2021-01-04 15:30
2021-01-04 14:12
2021-01-03 22:40
2021-01-01 14:28
2021-01-02 06:11
2021-01-05 00:19
2021-01-01 03:55
2021-01-01 01:21
2021-01-03 08:46
2021-01-02 00:55
2021-01-03 00:20
2021-01-05 21:09
2021-01-04 07:17
2021-01-02 18:48
2021-01-05 17:14
2021-01-01 00:30
2021-01-01 01:43
2021-01-04 11:19
2021-01-06 01:30
2021-01-05 18:47
2021-01-01 00:45
2021-01-01 01:00
2021-01-05 17:41
2021-01-05 20:21
2021-01-05 20:52
2021-01-05 22:23
2021-01-05 21:52
2021-01-06 16:44
2021-01-06 05:10
2021-01-01 05:09
2021-01-01 05:09
2021-01-06 13:59
2021-01-06 18:12
2021-01-05 21:30
2021-01-05 21:30
2021-01-05 21:30
2021-01-05 21:30
2021-01-06 15:30
2021-01-06 13:04
2021-01-07 02:

2021-04-27 07:50
2021-04-29 22:39
2021-04-29 22:39
2021-04-29 22:39
2021-04-29 22:39
2021-04-27 08:14
2021-05-01 08:48
2021-04-17 16:51
2021-04-23 19:13
2021-05-01 20:58
2021-04-28 12:02
2021-05-03 13:47
2021-05-03 16:35
2021-04-18 15:00
2021-05-02 19:39
2021-04-17 23:28
2021-04-23 14:00
2021-05-03 11:10
2021-05-02 01:22
2021-05-02 01:22
2021-05-02 01:22
2021-05-02 18:50
2021-05-02 18:50
2021-05-03 21:10
2021-04-28 17:20
2021-04-29 20:08
2021-04-29 20:17
2021-05-04 18:05
2021-05-04 18:05
2021-05-03 10:54
2021-04-28 21:50
2021-05-04 12:07
2021-05-05 00:25
2021-05-01 10:40
2021-05-05 02:13
2021-05-05 02:45
2021-04-30 21:02
2021-05-01 13:30
2021-05-05 13:05
2021-05-03 14:41
2021-05-02 22:21
2021-05-04 17:37
2021-04-20 00:15
2021-05-04 16:10
2021-05-04 16:10
2021-05-04 15:50
2021-05-04 15:50
2021-05-04 15:48
2021-05-03 18:59
2021-04-27 23:58
2021-04-30 21:30
2021-04-30 21:30
2021-05-05 18:40
2021-05-05 20:42
2021-05-04 15:59
2021-05-04 15:59
2021-05-04 20:52
2021-05-04 20:52
2021-05-04 20:

2021-08-17 07:30
2021-08-17 01:00
2021-08-17 15:45
2021-08-17 14:30
2021-08-17 21:20
2021-08-18 01:22
2021-08-18 00:50
2021-08-18 00:51
2021-08-18 10:26
2021-08-18 16:31
2021-08-18 23:27
2021-08-18 03:36
2021-08-18 22:07
2021-08-18 22:06
2021-08-18 22:06
2021-08-18 22:06
2021-08-18 19:27
2021-08-18 03:52
2021-08-18 15:20
2021-08-18 15:20
2021-08-18 01:14
2021-08-18 00:51
2021-08-18 11:01
2021-08-18 22:14
2021-08-18 14:59
2021-08-18 14:27
2021-08-18 23:29
2021-08-18 20:10
2021-08-18 09:57
2021-08-19 00:01
2021-08-19 13:26
2021-08-19 17:40
2021-08-19 17:40
2021-08-19 22:12
2021-08-19 17:30
2021-08-19 21:59
2021-08-19 22:22
2021-08-19 20:52
2021-08-19 00:57
2021-08-19 04:15
2021-08-19 10:33
2021-08-19 13:34
2021-08-19 21:35
2021-08-19 19:19
2021-08-19 14:56
2021-08-19 18:25
2021-08-19 01:11
2021-08-19 22:48
2021-08-19 14:25
2021-08-19 08:48
2021-08-19 13:37
2021-08-19 22:40
2021-08-19 01:20
2021-08-19 23:01
2021-08-19 19:24
2021-08-20 03:20
2021-08-20 12:09
2021-08-20 17:50
2021-08-20 01:

2021-11-13 23:07
2021-11-14 01:15
2021-11-14 01:15
2021-11-16 15:09
2021-11-16 19:39
2021-11-06 00:32
2021-11-10 22:26
2021-11-10 21:35
2021-11-10 21:35
2021-11-10 21:35
2021-11-13 05:34
2021-11-15 00:00
2021-11-13 00:42
2021-11-13 00:42
2021-11-12 16:21
2021-11-14 12:53
2021-11-10 21:30
2021-11-10 22:26
2021-11-16 00:00
2021-11-06 23:44
2021-11-12 23:14
2021-11-17 00:43
2021-11-16 06:27
2021-11-16 06:27
2021-11-16 09:30
2021-11-16 12:10
2021-11-16 01:07
2021-11-16 19:14
2021-11-17 01:10
2021-11-16 19:20
2021-11-15 12:56
2021-11-14 21:15
2021-10-05 01:01
2021-11-12 16:18
2021-11-12 16:18
2021-11-16 23:14
2021-11-17 00:07
2021-11-17 00:12
2021-11-17 00:53
2021-11-16 10:14
2021-10-19 19:18
2021-11-10 13:14
2021-11-16 08:45
2021-11-15 17:28
2021-11-15 17:28
2021-11-15 20:05
2021-11-15 20:05
2021-11-15 20:05
2021-11-15 16:52
2021-11-16 21:38
2021-11-02 14:15
2021-11-14 17:11
2021-11-05 14:30
2021-11-12 20:58
2021-11-12 20:58
2021-11-10 18:30
2021-11-13 20:38
2021-11-15 10:27
2021-11-13 21:

2020-01-30 23:49
2020-01-28 23:49
2020-01-31 18:12
2020-02-01 09:07
2020-01-31 21:12
2020-01-27 08:10
2020-01-28 21:50
2020-01-28 22:45
2020-01-28 22:45
2020-02-01 07:40
2020-02-01 07:45
2020-01-30 20:45
2020-01-05 20:48
2020-01-31 21:07
2020-02-01 19:15
2020-02-01 21:37
2020-02-01 22:12
2020-02-01 22:12
2020-01-16 09:03
2020-02-01 04:30
2020-02-01 04:30
2020-02-01 04:30
2020-02-01 04:30
2020-02-01 19:04
2020-01-31 19:58
2020-01-30 22:26
2020-02-01 05:20
2020-02-02 02:21
2020-01-23 14:00
2020-02-01 04:40
2020-01-23 22:39
2020-01-23 22:39
2020-01-25 17:35
2020-01-26 00:00
2020-01-28 08:35
2020-01-29 21:05
2020-01-28 21:34
2020-01-31 01:43
2020-01-30 02:50
2020-01-23 18:08
2020-02-02 02:57
2020-01-30 23:03
2020-01-30 20:18
2020-01-20 15:23
2020-01-20 15:23
2020-02-01 21:00
2020-02-02 10:25
2020-02-02 04:46
2020-01-25 22:45
2020-01-31 17:00
2020-01-02 01:11
2020-01-29 01:54
2020-01-29 00:12
2020-01-29 15:18
2020-02-02 19:33
2020-01-12 03:42
2020-01-28 08:37
2020-01-19 13:56
2020-01-24 22:

...

2018-09-27 21:00
2018-09-27 21:27
2018-09-27 03:34
2018-09-27 23:21
2018-09-27 18:23
2018-09-27 00:50
2018-09-27 00:50
2018-09-27 15:10
2018-09-27 15:10
2018-09-27 02:22
2018-09-27 16:45
2018-09-27 16:45
2018-09-27 16:45
2018-09-27 16:26
2018-09-27 23:22
2018-09-27 23:22
2018-09-27 23:26
2018-09-27 23:25
2018-09-27 22:00
2018-09-27 02:20
2018-09-27 09:26
2018-09-27 17:38
2018-09-27 20:37
2018-09-27 15:25
2018-09-27 23:48
2018-09-27 21:41
2018-09-27 21:41
2018-09-27 20:20
2018-09-27 18:58
2018-09-27 22:05
2018-09-27 22:05
2018-09-27 18:50
2018-09-28 14:38
2018-09-28 18:10
2018-09-28 18:10
2018-09-28 20:38
2018-09-28 17:40
2018-09-28 23:10
2018-09-28 02:15
2018-09-28 10:12
2018-09-28 10:10
2018-09-28 23:00
2018-09-28 20:02
2018-09-28 20:02
2018-09-28 18:25
2018-09-28 20:10
2018-09-28 17:00
2018-09-28 21:30
2018-09-28 14:18
2018-09-28 14:18
2018-09-28 14:18
2018-09-28 18:25
2018-09-28 10:02
2018-09-28 20:03
2018-09-28 20:10
2018-09-28 22:40
2018-09-28 22:40
2018-09-28 20:05
2018-09-28 15:

2018-12-28 17:13
2018-12-28 02:46
2018-12-28 02:46
2018-12-28 02:46
2018-12-28 15:08
2018-12-28 15:08
2018-12-28 15:08
2018-12-28 13:10
2018-12-28 22:15
2018-12-28 00:20
2018-12-28 20:08
2018-12-28 20:29
2018-12-28 15:10
2018-12-28 21:45
2018-12-28 01:32
2018-12-28 00:39
2018-12-28 16:38
2018-12-28 19:31
2018-12-28 23:45
2018-12-28 23:45
2018-12-28 03:05
2018-12-28 20:05
2018-12-28 01:56
2018-12-28 23:20
2018-12-28 20:35
2018-12-29 23:12
2018-12-29 04:11
2018-12-29 18:56
2018-12-29 09:50
2018-12-29 16:20
2018-12-29 00:40
2018-12-29 10:55
2018-12-29 13:49
2018-12-29 13:49
2018-12-29 13:49
2018-12-29 17:55
2018-12-29 12:14
2018-12-29 11:43
2018-12-29 13:37
2018-12-29 11:06
2018-12-29 00:55
2018-12-29 17:06
2018-12-29 05:05
2018-12-29 05:04
2018-12-29 08:00
2018-12-29 15:10
2018-12-29 17:26
2018-12-29 16:20
2018-12-29 05:47
2018-12-29 05:12
2018-12-29 15:12
2018-12-29 15:12
2018-12-29 15:12
2018-12-29 12:54
2018-12-29 23:26
2018-12-29 22:00
2018-12-30 14:25
2018-12-30 11:58
2018-12-30 01:

2017-05-02 09:35
2017-05-04 01:04
2017-05-04 01:35
2017-05-03 19:49
2017-05-05 09:19
2017-05-06 15:14
2017-05-07 05:29
2017-05-08 21:35
2017-05-08 21:35
2017-05-08 21:35
2017-05-05 19:49
2017-05-06 00:10
2017-05-06 00:10
2017-05-04 15:14
2017-05-04 16:15
2017-05-10 10:48
2017-05-11 18:28
2017-05-12 01:51
2017-05-13 08:52
2017-05-15 06:05
2017-05-15 06:05
2017-05-15 13:10
2017-05-17 17:00
2017-05-18 18:19
2017-05-18 17:54
2017-05-18 17:54
2017-05-21 21:09
2017-03-23 01:53
2017-05-25 01:56
2017-05-27 22:39
2017-05-27 22:40
2017-05-27 22:39
2017-05-22 08:18
2017-05-31 02:02
2017-05-31 21:10
2017-06-01 01:20
2017-06-01 20:05
2017-05-31 17:29
2017-04-25 01:58
2017-06-02 23:20
2017-06-03 04:20
2017-06-03 04:17
2017-05-31 15:59
2017-06-05 11:50
2017-06-05 10:49
2017-06-06 21:14
2017-05-30 11:09
2017-06-08 12:55
2017-06-08 16:24
2017-06-08 19:04
2017-06-09 00:39
2017-06-06 13:35
2017-06-06 13:35
2017-06-09 22:51
2017-06-09 20:20
2017-06-09 21:19
2017-06-10 03:59
2017-06-10 03:59
2017-06-10 19:

2017-09-17 20:00
2017-09-17 20:00
2017-09-29 18:05
2017-10-07 05:11
2017-10-12 01:15
2017-10-13 13:56
2017-10-13 13:56
2017-11-01 12:34
2017-10-31 21:55
2017-10-31 21:55
2017-10-31 21:55
2017-10-31 21:55
2017-10-31 21:55
2017-11-05 12:25
2017-11-18 03:35
2017-11-18 03:35
2017-11-18 19:20
2017-11-27 00:17
2017-11-27 00:17
2017-11-27 13:10
2017-11-30 12:50
2017-12-03 02:34
2017-12-17 06:25
2017-12-17 06:25
2017-12-20 18:21
2017-12-20 21:30
2017-01-04 11:42
2017-01-12 23:05
2017-01-12 23:05
2017-01-05 15:30
2017-01-27 23:00
2017-01-27 23:00
2017-01-27 22:53
2017-01-27 22:53
2017-02-02 23:47
2017-02-02 23:47
2017-02-02 23:47
2017-02-02 23:47
2017-02-07 01:04
2017-02-19 23:50
2017-02-24 02:09
2017-02-16 20:50
2017-02-25 18:23
2017-02-27 11:41
2017-02-24 02:07
2017-03-02 13:06
2017-02-24 02:09
2017-02-24 02:09
2017-02-24 02:09
2017-02-24 02:09
2017-02-24 02:09
2017-02-24 02:09
2017-02-24 02:09
2017-02-24 02:09
2017-02-09 02:11
2017-03-04 09:43
2017-03-02 11:15
2017-03-02 11:15
2017-03-02 11:

2017-02-23 17:25
2017-03-03 08:54
2017-03-27 17:50
2017-04-09 13:51
2017-04-11 22:30
2017-04-12 03:30
2017-04-16 01:42
2017-04-16 01:42
2017-04-16 01:42
2017-04-21 23:40
2017-04-21 23:15
2017-05-02 20:45
2017-05-02 20:45
2017-05-02 20:45
2017-05-03 06:10
2017-05-03 12:57
2017-05-03 12:57
2017-05-03 12:57
2017-05-03 12:57
2017-05-03 12:57
2017-05-06 04:53
2017-05-06 04:53
2017-05-24 03:01
2017-06-03 02:51
2017-06-03 02:51
2017-06-05 06:21
2017-06-06 09:48
2017-06-06 09:48
2017-06-06 09:48
2017-06-06 09:48
2017-06-06 09:48
2017-06-03 20:00
2017-03-13 11:36
2017-06-13 16:57
2017-06-13 16:57
2017-06-27 21:19
2017-06-27 21:19
2017-06-27 21:19
2017-06-27 21:19
2017-07-05 03:01
2017-07-15 00:59
2017-07-15 00:59
2017-07-15 00:59
2017-03-11 13:58
2017-03-23 12:28
2017-04-07 11:47
2017-04-08 01:44
2017-03-11 00:35
2017-03-04 21:35
2017-04-18 19:24
2017-08-01 16:04
2017-08-04 21:30
2017-08-04 21:30
2017-08-06 02:29
2017-08-07 15:17
2017-08-09 04:04
2017-08-17 17:00
2017-08-19 12:23
2017-08-19 12:

In [21]:
for x in df_1['DATE_TIME']:
    if ':' not in x:
        x_replace = x + "00:00"
        df_1.replace(x, x_replace, inplace = True)

In [22]:
for x in df_1['DATE_TIME']:
    if ':' not in x:
        print(x)

In [23]:
pd.to_datetime(df_1['DATE_TIME'], format = '%Y-%m-%d %H:%M')
df_1.set_index('DATE_TIME', drop = True, inplace = True)
df_1.info()

# putting the datetime as the index

<class 'pandas.core.frame.DataFrame'>
Index: 54587 entries, 2021-01-01 01:50 to 2017-11-12 22:55
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   SUSPECTED_CRIME_DESCRIPTION  54587 non-null  object 
 1   SUSPECT_ARRESTED_FLAG        54587 non-null  object 
 2   SUSPECT_RACE_DESCRIPTION     54587 non-null  object 
 3   PRECINCT                     54156 non-null  float64
dtypes: float64(1), object(3)
memory usage: 2.1+ MB


In [24]:
before_mapping_shape = df_1.shape

# making sure the shapes are the same

In [25]:
for x in df_1.index:
    print(x)

2021-01-01 01:50
2021-01-01 10:55
2021-01-01 10:10
2021-01-01 20:17
2021-01-01 19:59
2021-01-01 21:29
2021-01-01 16:33
2021-01-02 20:48
2021-01-02 19:50
2021-01-02 22:34
2021-01-02 22:12
2021-01-02 19:15
2021-01-01 22:20
2021-01-01 16:10
2021-01-02 14:56
2021-01-04 04:24
2021-01-03 23:41
2021-01-04 09:06
2021-01-04 09:09
2021-01-04 15:30
2021-01-04 14:12
2021-01-03 22:40
2021-01-01 14:28
2021-01-02 06:11
2021-01-05 00:19
2021-01-01 03:55
2021-01-01 01:21
2021-01-03 08:46
2021-01-02 00:55
2021-01-03 00:20
2021-01-05 21:09
2021-01-04 07:17
2021-01-02 18:48
2021-01-05 17:14
2021-01-01 00:30
2021-01-01 01:43
2021-01-04 11:19
2021-01-06 01:30
2021-01-05 18:47
2021-01-01 00:45
2021-01-01 01:00
2021-01-05 17:41
2021-01-05 20:21
2021-01-05 20:52
2021-01-05 22:23
2021-01-05 21:52
2021-01-06 16:44
2021-01-06 05:10
2021-01-01 05:09
2021-01-01 05:09
2021-01-06 13:59
2021-01-06 18:12
2021-01-05 21:30
2021-01-05 21:30
2021-01-05 21:30
2021-01-05 21:30
2021-01-06 15:30
2021-01-06 13:04
2021-01-07 02:

2021-05-19 14:41
2021-05-17 15:25
2021-05-18 17:14
2021-05-07 02:39
2021-04-25 21:24
2021-05-19 01:19
2021-05-18 21:30
2021-05-18 21:20
2021-05-18 07:37
2021-05-18 20:42
2021-05-18 20:42
2021-05-20 00:00
2021-05-07 22:50
2021-05-20 00:25
2021-05-20 00:25
2021-05-19 14:56
2021-05-19 18:04
2021-05-18 13:54
2021-05-17 22:25
2021-05-18 12:22
2021-05-15 14:36
2021-05-11 19:58
2021-05-19 07:20
2021-05-14 02:30
2021-05-19 22:56
2021-05-14 14:15
2021-05-17 22:35
2021-05-18 09:18
2021-05-07 17:35
2021-05-20 19:05
2021-05-20 19:05
2021-05-20 19:05
2021-05-17 02:47
2021-05-20 12:15
2021-05-20 17:58
2021-05-18 20:22
2021-05-20 20:55
2021-05-20 21:30
2021-05-20 21:30
2021-05-12 08:17
2021-05-19 05:43
2021-05-17 20:07
2021-05-11 23:20
2021-05-19 01:52
2021-05-19 18:44
2021-05-18 14:37
2021-05-21 11:42
2021-05-19 13:43
2021-05-15 14:15
2021-05-14 22:05
2021-05-15 19:56
2021-05-05 09:02
2021-05-05 09:15
2021-05-18 01:46
2021-05-18 00:42
2021-05-17 23:03
2021-05-19 10:45
2021-05-19 10:45
2021-05-20 16:

2021-09-02 12:21
2021-09-03 00:24
2021-09-03 18:21
2021-09-03 00:20
2021-09-03 11:08
2021-09-03 21:00
2021-09-03 19:20
2021-09-03 19:14
2021-09-03 15:11
2021-09-03 00:57
2021-09-03 00:57
2021-09-03 03:00
2021-09-03 01:51
2021-09-03 19:51
2021-09-03 21:45
2021-09-03 01:40
2021-09-03 12:18
2021-09-03 20:42
2021-09-03 23:25
2021-09-04 14:00
2021-09-04 22:34
2021-09-04 03:45
2021-09-04 17:00
2021-09-04 17:00
2021-09-04 07:05
2021-09-04 13:40
2021-09-04 20:28
2021-09-04 03:31
2021-09-04 21:00
2021-09-04 00:24
2021-09-04 22:55
2021-09-04 21:25
2021-09-04 01:14
2021-09-04 20:18
2021-09-04 20:18
2021-09-04 16:31
2021-09-04 16:31
2021-09-04 21:20
2021-09-04 00:43
2021-09-04 00:43
2021-09-04 00:43
2021-09-04 13:19
2021-09-04 13:19
2021-09-04 23:03
2021-09-04 11:03
2021-09-04 16:12
2021-09-04 00:53
2021-09-04 22:53
2021-09-04 18:40
2021-09-04 20:24
2021-09-04 20:24
2021-09-04 20:25
2021-09-04 20:24
2021-09-04 20:24
2021-09-04 23:10
2021-09-04 15:20
2021-09-04 02:26
2021-09-04 19:10
2021-09-04 19:

2021-12-08 16:21
2021-12-08 20:17
2021-12-09 02:12
2021-12-07 17:31
2021-12-04 20:06
2021-12-07 23:10
2021-12-09 20:47
2021-11-30 21:43
2021-12-05 02:35
2021-12-09 01:40
2021-12-09 10:56
2021-12-09 00:02
2021-12-10 00:08
2021-12-07 19:30
2021-12-10 03:33
2021-11-29 14:20
2021-12-05 09:25
2021-12-05 09:23
2021-12-10 09:46
2021-12-03 07:30
2021-12-08 18:32
2021-12-08 18:32
2021-12-09 16:04
2021-12-09 18:40
2021-11-25 16:15
2021-12-09 21:54
2021-12-09 21:53
2021-12-09 08:37
2021-11-28 22:10
2021-11-28 22:10
2021-12-01 20:31
2021-12-08 18:26
2021-12-04 21:28
2021-12-09 00:00
2021-12-10 11:35
2021-12-08 22:58
2021-12-10 12:21
2021-12-08 20:04
2021-12-09 16:51
2021-12-05 03:09
2021-12-05 03:09
2021-12-09 20:05
2021-12-09 20:05
2021-12-10 08:40
2021-12-10 20:10
2021-12-10 17:50
2021-12-10 17:50
2021-12-02 23:10
2021-12-09 01:19
2021-12-09 06:43
2021-12-09 06:43
2021-12-10 20:38
2021-12-10 12:34
2021-12-10 12:34
2021-12-10 12:34
2021-12-10 12:34
2021-12-10 12:34
2021-12-10 12:34
2021-12-09 11:

2020-03-07 23:00
2020-03-09 16:22
2020-03-09 09:00
2020-03-06 20:23
2020-03-09 16:44
2020-03-09 18:29
2020-03-02 19:25
2020-03-08 21:00
2020-02-14 18:28
2020-03-09 20:40
2020-03-04 16:10
2020-03-09 06:45
2020-03-09 12:26
2020-03-09 12:26
2020-02-29 10:58
2020-03-03 09:25
2020-03-07 02:56
2020-03-08 21:40
2020-03-10 00:14
2020-03-10 04:25
2020-03-05 22:57
2020-02-20 23:20
2020-02-29 03:00
2020-02-29 03:00
2020-02-26 20:45
2020-03-08 03:36
2020-03-08 03:36
2020-03-08 03:37
2020-03-02 22:21
2020-03-09 11:00
2020-03-10 12:22
2020-03-09 17:50
2020-03-10 10:30
2020-03-05 18:36
2020-03-05 01:20
2020-03-02 22:50
2020-02-26 20:45
2020-03-03 14:26
2020-03-03 14:26
2020-03-04 22:27
2020-03-04 22:27
2020-03-04 22:27
2020-03-10 17:31
2020-03-06 17:37
2020-03-06 17:35
2020-02-23 19:27
2020-03-10 16:21
2020-03-10 17:25
2020-02-26 18:19
2020-03-07 21:50
2020-03-10 15:29
2020-03-05 21:10
2020-03-08 04:15
2020-03-08 01:54
2020-03-08 01:54
2020-02-28 21:08
2020-02-27 16:35
2020-03-10 19:42
2020-03-09 19:

...

2017-08-25 19:49
2017-08-25 19:49
2017-08-31 11:23
2017-09-03 02:11
2017-09-08 23:54
2017-09-08 22:49
2017-09-13 23:14
2017-09-13 22:06
2017-09-12 09:19
2017-09-21 16:40
2017-10-03 19:44
2017-10-06 09:18
2017-10-06 09:18
2017-10-06 09:18
2017-10-06 09:18
2017-10-06 09:18
2017-10-05 15:59
2017-10-12 01:55
2017-10-19 23:45
2017-10-29 04:03
2017-10-29 04:03
2017-11-10 19:30
2017-11-15 17:43
2017-11-17 01:06
2017-11-17 01:21
2017-11-28 21:45
2017-12-01 21:18
2017-10-13 23:41
2017-11-17 17:09
2017-11-17 17:05
2017-12-08 14:09
2017-12-08 21:59
2017-12-16 22:08
2017-12-16 22:08
2017-12-16 21:03
2017-12-16 21:03
2017-12-16 21:03
2017-12-16 21:03
2017-12-16 21:03
2017-12-20 22:24
2017-12-26 18:35
2017-12-28 21:19
2017-01-08 19:02
2017-01-08 19:02
2017-01-10 01:30
2017-01-06 19:30
2017-01-09 11:00
2017-01-09 11:15
2017-01-12 16:49
2017-01-11 11:09
2017-01-11 11:09
2017-01-10 22:47
2017-01-13 20:34
2017-01-16 17:00
2017-01-18 18:30
2017-01-19 21:30
2017-01-19 21:44
2017-01-20 23:07
2017-01-23 15:

2017-03-04 02:34
2017-03-04 02:34
2017-03-04 03:50
2017-03-08 21:37
2017-03-10 20:40
2017-03-14 22:24
2017-03-10 22:44
2017-03-17 19:30
2017-03-21 14:09
2017-03-22 21:39
2017-03-25 16:49
2017-03-27 20:45
2017-03-27 20:45
2017-03-27 12:39
2017-04-01 00:21
2017-04-05 21:43
2017-04-08 09:21
2017-04-08 20:18
2017-04-08 20:18
2017-04-08 20:18
2017-04-13 15:14
2017-04-13 15:14
2017-03-13 15:14
2017-04-18 20:09
2017-04-18 20:09
2017-04-19 11:09
2017-04-20 20:06
2017-04-21 17:34
2017-04-30 01:57
2017-05-05 04:10
2017-05-06 20:29
2017-05-04 06:30
2017-05-04 06:30
2017-05-09 07:11
2017-05-09 23:54
2017-05-09 19:58
2017-04-27 23:40
2017-05-15 09:50
2017-05-18 23:45
2017-05-19 23:54
2017-05-19 23:54
2017-05-21 21:50
2017-01-26 04:30
2017-05-24 17:09
2017-05-26 18:30
2017-05-27 09:44
2017-05-31 23:00
2017-05-31 23:29
2017-06-01 23:29
2017-06-03 23:20
2017-06-03 23:20
2017-06-03 23:20
2017-06-04 14:09
2017-06-05 20:00
2017-06-06 01:57
2017-06-08 00:00
2017-06-09 04:31
2017-06-10 00:10
2017-06-10 00:

2017-06-11 08:29
2017-06-11 08:29
2017-06-26 20:42
2017-06-28 22:49
2017-06-29 22:38
2017-07-03 22:44
2017-07-03 22:44
2017-07-10 02:49
2017-07-13 09:57
2017-07-13 09:59
2017-07-13 19:10
2017-07-14 16:12
2017-07-17 11:56
2017-07-19 02:29
2017-07-22 20:58
2017-07-22 20:58
2017-07-22 20:58
2017-07-23 07:42
2017-07-24 13:56
2017-07-25 23:07
2017-07-27 21:23
2017-07-30 17:56
2017-08-05 02:25
2017-08-07 18:59
2017-08-07 18:59
2017-08-12 13:02
2017-08-12 13:02
2017-08-12 13:02
2017-08-12 13:02
2017-08-13 00:19
2017-08-14 12:59
2017-08-14 12:59
2017-08-14 12:59
2017-08-18 23:54
2017-08-19 18:59
2017-08-19 18:59
2017-08-19 18:59
2017-08-21 02:09
2017-08-25 02:15
2017-08-25 02:15
2017-08-25 02:15
2017-08-31 23:09
2017-08-31 00:19
2017-09-02 01:15
2017-09-01 00:45
2017-08-31 23:50
2017-09-04 19:40
2017-09-04 19:40
2017-09-06 08:02
2017-08-07 01:35
2017-09-08 12:14
2017-08-31 12:10
2017-09-17 13:10
2017-09-17 17:02
2017-09-25 20:00
2017-09-28 13:10
2017-09-28 13:46
2017-10-01 19:40
2017-10-01 19:

2017-02-28 13:20
2017-02-28 13:20
2017-03-07 15:03
2017-03-07 15:05
2017-03-07 20:11
2017-03-04 01:10
2017-03-04 01:10
2017-03-09 00:10
2017-03-08 18:01
2017-03-08 18:01
2017-03-10 16:24
2017-03-10 23:18
2017-03-21 15:23
2017-03-27 13:44
2017-04-03 17:16
2017-04-06 21:35
2017-01-08 03:12
2017-04-09 17:45
2017-04-20 08:20
2017-04-22 20:24
2017-03-31 02:20
2017-04-27 01:37
2017-04-29 22:10
2017-04-29 22:10
2017-04-29 22:10
2017-04-29 22:10
2017-04-29 22:10
2017-04-29 22:10
2017-04-29 22:10
2017-04-30 15:19
2017-04-30 15:19
2017-05-07 11:50
2017-05-11 11:29
2017-05-16 13:04
2017-05-20 17:11
2017-05-22 00:39
2017-05-25 20:16
2017-05-26 17:05
2017-06-01 23:29
2017-06-02 22:31
2017-06-05 23:15
2017-06-05 01:44
2017-06-05 16:15
2017-06-05 16:15
2017-06-06 11:09
2017-06-06 11:18
2017-06-06 11:18
2017-06-06 21:39
2017-06-06 21:39
2017-06-06 21:39
2017-06-06 21:39
2017-06-07 02:20
2017-06-07 23:54
2017-06-03 19:15
2017-06-11 09:30
2017-06-11 09:30
2017-06-14 15:55
2017-06-21 21:19
2017-06-26 19:

2017-01-22 16:04
2017-01-22 19:49
2017-02-08 05:20
2017-01-23 16:58
2017-01-23 16:58
2017-02-06 14:20
2017-02-08 13:04
2017-02-08 13:04
2017-02-25 07:08
2017-02-27 16:30
2017-02-27 16:30
2017-02-21 22:04
2017-02-21 22:04
2017-02-21 22:04
2017-02-24 17:58
2017-02-24 17:58
2017-02-24 17:58
2017-02-24 17:58
2017-03-20 18:55
2017-03-26 17:18
2017-04-15 19:30
2017-04-15 19:35
2017-04-15 19:35
2017-04-21 05:06
2017-04-29 18:59
2017-04-29 18:59
2017-04-29 18:59
2017-05-02 04:42
2017-05-02 23:45
2017-05-08 04:49
2017-05-16 08:34
2017-05-16 09:35
2017-05-16 09:35
2017-05-23 17:00
2017-05-23 17:00
2017-05-24 16:24
2017-06-01 12:35
2017-06-01 12:35
2017-06-03 00:00
2017-06-03 16:10
2017-06-03 16:10
2017-06-05 14:51
2017-04-15 19:35
2017-06-12 14:00
2017-06-22 17:25
2017-06-22 17:25
2017-06-03 16:53
2017-06-29 17:54
2017-06-29 15:55
2017-05-03 10:04
2017-03-10 09:16
2017-03-10 09:16
2017-07-14 22:24
2017-07-14 20:20
2017-07-14 20:20
2017-07-14 20:20
2017-07-19 18:50
2017-07-28 11:15
2017-07-01 17:

### Cleaning up the Race columns in first df

I'll be changing "MIDDLE EASTERN" to "W. ASIAN" as it's more precise. 

citation on "West Asia" https://www.businessinsider.com/why-you-should-stop-calling-it-the-middle-east-2015-1


In [26]:
# making a dictionary to make null and blank and "male" races into "unknown"
# combining some similar minority races into 'E. ASIAN' or 'NATIVE' respectively

race_map = {'BLACK': 'BLACK', 'WHITE HISPANIC' : 'WHITE HISPANIC', 'WHITE' : 'WHITE', 
 'BLACK HISPANIC' : 'BLACK HISPANIC', 'ASIAN / PACIFIC ISLANDER' : 'E. ASIAN', 
 'ASIAN/PAC.ISL' : 'E. ASIAN', 'MIDDLE EASTERN/SOUTHWEST' : 'W. ASIAN', 
 'AMERICAN INDIAN/ALASKAN N' : 'NATIVE', 
 'AMERICAN INDIAN/ALASKAN NATIVE' : 'NATIVE', 'AMER IND' : 'NATIVE', 
 'MALE' : 'UNKNOWN', '' : 'UNKNOWN', '(null)' : 'UNKNOWN'} 

In [27]:
# mapping the races to the dictionary for new values

df_1['SUSPECT_RACE_DESCRIPTION'] = df_1['SUSPECT_RACE_DESCRIPTION'].map(race_map)

In [28]:
df_1['SUSPECT_RACE_DESCRIPTION'].value_counts()

# inspecting the changes

BLACK             31588
WHITE HISPANIC    11486
WHITE              4854
BLACK HISPANIC     4675
E. ASIAN           1139
UNKNOWN             584
W. ASIAN            132
NATIVE               58
Name: SUSPECT_RACE_DESCRIPTION, dtype: int64

In [29]:
df_1.shape == before_mapping_shape

# making sure we didn't lose rows

True

## second df 2017-2011

In [30]:
# making a dictionary that we can keep calling on to make sure the rest of 
# the data loads into appropriate types (since there's mixed data types in 
# original data set)

data_type_dict = {'datestop' : 'str',
'timestop' : 'str',
'arstmade' : 'str',
'race' : 'str',
'pct' : 'str',
'crimsusp' : 'str'}

In [31]:
# making a dictionary that will rename the older columns to match with their 
# corrosponding modern equivilants 

rename_columns_dict = {'arstmade' : 'SUSPECT_ARRESTED_FLAG',
                       'race': 'SUSPECT_RACE_DESCRIPTION',
                       'pct' : 'PRECINCT',
                      'crimsusp' : 'SUSPECTED_CRIME_DESCRIPTION'}

In [32]:
#df_sixteen = pd.read_csv('SF_2016.csv')

#df_sixteen.to_csv("SF_2016.csv.gz", compression = 'gzip', index = False)

# loading in 2016 data, while parsing the dates, shrinking the data and 
# reloading the data, and sending the datetime column through all the reformating

df_sixteen = pd.read_csv("raw_data/SF_2016.csv.gz", dtype=data_type_dict)
df_sixteen = format_csv_dfs(df_sixteen)
df_sixteen['DATE_TIME'] = df_sixteen['datestop'] + ' ' + df_sixteen['timestop']

# removing redundant columns 

columns_to_drop = ['datestop', 'timestop']

df_sixteen.rename(columns = rename_columns_dict, inplace = True)
df_sixteen.drop(columns = columns_to_drop, inplace = True)

/Users/b0ihazard/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (0,2,8,18,19,73,74,84,85,86,103,110,111) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [33]:
df_sixteen.tail()

,PRECINCT,SUSPECTED_CRIME_DESCRIPTION,SUSPECT_ARRESTED_FLAG,SUSPECT_RACE_DESCRIPTION,DATE_TIME
12400,123,FEL,N,W,12022016 1847
12401,123,MISD,N,U,12022016 255
12402,123,MISD,Y,U,12042016 2225
12403,123,MISD,N,W,11222016 2212
12404,,,,,


In [34]:
df_sixteen = df_sixteen.drop(index = 12404)

In [35]:
df_sixteen['DATE_TIME'] = reformat_date_time(df_sixteen)

In [36]:
df_sixteen['DATE_TIME']

0        02/07/2016 01:00
1        02/18/2016 00:30
2        01/01/2016 00:30
3        01/01/2016 00:40
4        01/01/2016 00:50
               ...       
12399    11/26/2016 15:30
12400    12/02/2016 18:47
12401    12/02/2016 02:55
12402    12/04/2016 22:25
12403    11/22/2016 22:12
Name: DATE_TIME, Length: 12404, dtype: object

In [37]:
df_sixteen.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12404 entries, 0 to 12403
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   PRECINCT                     12404 non-null  object
 1   SUSPECTED_CRIME_DESCRIPTION  12401 non-null  object
 2   SUSPECT_ARRESTED_FLAG        12404 non-null  object
 3   SUSPECT_RACE_DESCRIPTION     12404 non-null  object
 4   DATE_TIME                    12404 non-null  object
dtypes: object(5)
memory usage: 581.4+ KB


In [38]:
#df_fifteen = pd.read_csv('SF_2015.csv')

#df_fifteen.to_csv("SF_2015.csv.gz", compression = 'gzip', index = False)

# loading in 2015 data, while parsing the dates, shrinking the data and 
# reloading the data, and sending the datetime column through all the reformating

df_fifteen = pd.read_csv("raw_data/SF_2015.csv.gz", dtype=data_type_dict)
df_fifteen = format_csv_dfs(df_fifteen)
df_fifteen['DATE_TIME'] = df_fifteen['datestop'] + ' ' + df_fifteen['timestop']

# removing redundant columns 

columns_to_drop = ['datestop', 'timestop']

df_fifteen.rename(columns = rename_columns_dict, inplace = True)
df_fifteen.drop(columns = columns_to_drop, inplace = True)

In [39]:
df_fifteen['DATE_TIME'] 

0          1012015 315
1         1152015 1747
2         1292015 1745
3         1292015 1745
4         1292015 1745
             ...      
22558    12312015 2250
22559    12312015 2305
22560    12312015 2305
22561    12312015 2345
22562    12312015 2358
Name: DATE_TIME, Length: 22563, dtype: object

In [40]:
df_fifteen['DATE_TIME'] = reformat_date_time(df_fifteen)

In [41]:
df_fifteen['DATE_TIME']

0        01/01/2015 03:15
1        01/15/2015 17:47
2        01/29/2015 17:45
3        01/29/2015 17:45
4        01/29/2015 17:45
               ...       
22558    12/31/2015 22:50
22559    12/31/2015 23:05
22560    12/31/2015 23:05
22561    12/31/2015 23:45
22562    12/31/2015 23:58
Name: DATE_TIME, Length: 22563, dtype: object

In [42]:
df_fifteen.head()
# inspecting the data

,PRECINCT,SUSPECTED_CRIME_DESCRIPTION,SUSPECT_ARRESTED_FLAG,SUSPECT_RACE_DESCRIPTION,DATE_TIME
0,61,FELONY,N,W,01/01/2015 03:15
1,22,FELONY,N,B,01/15/2015 17:47
2,20,MISD,N,B,01/29/2015 17:45
3,20,MIDS,N,B,01/29/2015 17:45
4,20,MISD,N,B,01/29/2015 17:45


In [43]:
#df_fourteen = pd.read_csv('SF_2014.csv')

#df_fourteen.to_csv("SF_2014.csv.gz", compression = 'gzip', index = False)

# loading in 2014 data, while parsing the dates, shrinking the data and 
# reloading the data, and sending the datetime column through all the reformating

df_fourteen = pd.read_csv("raw_data/SF_2014.csv.gz", dtype=data_type_dict)
df_fourteen = format_csv_dfs(df_fourteen)
df_fourteen['DATE_TIME'] = df_fourteen['datestop'] + ' ' + df_fourteen['timestop']

# removing redundant columns 

columns_to_drop = ['datestop', 'timestop']

df_fourteen.rename(columns = rename_columns_dict, inplace = True)
df_fourteen.drop(columns = columns_to_drop, inplace = True)

/Users/b0ihazard/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10,83) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [44]:
df_fourteen['DATE_TIME']
# inspecting the data

0         1012014 2330
1         1032014 1530
2         1042014 2100
3         1092014 1250
4         1092014 1310
             ...      
45782    12312014 1838
45783    12312014 1910
45784    12312014 2016
45785    12312014 2320
45786    12312014 2342
Name: DATE_TIME, Length: 45787, dtype: object

In [45]:
df_fourteen['DATE_TIME'] = reformat_date_time(df_fourteen)

In [46]:
df_fourteen['DATE_TIME']

0        01/01/2014 23:30
1        01/03/2014 15:30
2        01/04/2014 21:00
3        01/09/2014 12:50
4        01/09/2014 13:10
               ...       
45782    12/31/2014 18:38
45783    12/31/2014 19:10
45784    12/31/2014 20:16
45785    12/31/2014 23:20
45786    12/31/2014 23:42
Name: DATE_TIME, Length: 45787, dtype: object

In [47]:
#df_thirteen = pd.read_csv('SF_2013.csv')

#df_thirteen.to_csv("SF_2013.csv.gz", compression = 'gzip', index = False)

# loading in 2013 data, while parsing the dates, shrinking the data and 
# reloading the data, and sending the datetime column through all the reformating

df_thirteen = pd.read_csv("raw_data/SF_2013.csv.gz", dtype=data_type_dict)
df_thirteen = format_csv_dfs(df_thirteen)
df_thirteen['DATE_TIME'] = df_thirteen['datestop'] + ' ' + df_thirteen['timestop']

# removing redundant columns 

columns_to_drop = ['datestop', 'timestop']

df_thirteen.rename(columns = rename_columns_dict, inplace = True)
df_thirteen.drop(columns = columns_to_drop, inplace = True)

/Users/b0ihazard/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10,73,83) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [48]:
df_thirteen['DATE_TIME']
# inspecting the data

0           1012013 452
1           1012013 315
2           1012013 300
3          1012013 1212
4          1012013 2300
              ...      
191846    12312013 2320
191847    12312013 2337
191848    12312013 2350
191849    12312013 2350
191850    12312013 2350
Name: DATE_TIME, Length: 191851, dtype: object

In [49]:
df_thirteen['DATE_TIME'] = reformat_date_time(df_thirteen)

In [50]:
df_thirteen['DATE_TIME']

0         01/01/2013 04:52
1         01/01/2013 03:15
2         01/01/2013 03:00
3         01/01/2013 12:12
4         01/01/2013 23:00
                ...       
191846    12/31/2013 23:20
191847    12/31/2013 23:37
191848    12/31/2013 23:50
191849    12/31/2013 23:50
191850    12/31/2013 23:50
Name: DATE_TIME, Length: 191851, dtype: object

In [51]:
# df_twelve = pd.read_csv('SF_2012.csv')

# df_twelve.to_csv("SF_2012.csv.gz", compression = 'gzip', index = False)

# loading in 2012 data, while parsing the dates, shrinking the data and 
# reloading the data, and sending the datetime column through all the reformating

df_twelve = pd.read_csv("raw_data/SF_2012.csv.gz", dtype=data_type_dict)
df_twelve = format_csv_dfs(df_twelve)
df_twelve['DATE_TIME'] = df_twelve['datestop'] + ' ' + df_twelve['timestop']

# removing redundant columns 

columns_to_drop = ['datestop', 'timestop']

df_twelve.rename(columns = rename_columns_dict, inplace = True)
df_twelve.drop(columns = columns_to_drop, inplace = True)

/Users/b0ihazard/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10,73,103,110,111) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [52]:
df_twelve['DATE_TIME']
# inspecting the data

0           1012012 115
1           1012012 310
2          1012012 2000
3          1022012 1245
4          1042012 2220
              ...      
532906    11042012 1930
532907    11042012 1930
532908    11042012 2305
532909    11042012 2305
532910    11082012 2113
Name: DATE_TIME, Length: 532911, dtype: object

In [53]:
df_twelve['DATE_TIME'] = reformat_date_time(df_twelve)

In [54]:
df_twelve['DATE_TIME']

0         01/01/2012 01:15
1         01/01/2012 03:10
2         01/01/2012 20:00
3         01/02/2012 12:45
4         01/04/2012 22:20
                ...       
532906    11/04/2012 19:30
532907    11/04/2012 19:30
532908    11/04/2012 23:05
532909    11/04/2012 23:05
532910    11/08/2012 21:13
Name: DATE_TIME, Length: 532911, dtype: object

In [55]:
#df_eleven = pd.read_csv(r'SF_2011.csv', 
# encoding = 'latin_1')

#latin_1 encoding needed due to some impropertly escaped charecters in file

#df_eleven.to_csv("SF_2011.csv.gz", compression = 'gzip', index = False)

# loading in 2012 data, while parsing the dates, shrinking the data and 
# reloading the data, and sending the datetime column through all the reformating

df_eleven = pd.read_csv("raw_data/SF_2011.csv.gz", dtype=data_type_dict)
df_eleven = format_csv_dfs(df_eleven)
df_eleven['DATE_TIME'] = df_eleven['datestop'] + ' ' + df_eleven['timestop']

# removing redundant columns 

columns_to_drop = ['datestop', 'timestop']

df_eleven.rename(columns = rename_columns_dict, inplace = True)
df_eleven.drop(columns = columns_to_drop, inplace = True)

/Users/b0ihazard/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (73,103) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [56]:
df_eleven['DATE_TIME']
# inspecting the data

0             1012011 0
1             1012011 5
2             1012011 7
3             1012011 7
4             1012011 7
              ...      
685719    12312011 2355
685720    12312011 2358
685721    12312011 2358
685722    12312011 2358
685723    12312011 2358
Name: DATE_TIME, Length: 685724, dtype: object

In [57]:
df_eleven['DATE_TIME'] = reformat_date_time(df_eleven)

In [58]:
df_eleven['DATE_TIME']

0         01/01/2011 00:00
1         01/01/2011 05:00
2         01/01/2011 07:00
3         01/01/2011 07:00
4         01/01/2011 07:00
                ...       
685719    12/31/2011 23:55
685720    12/31/2011 23:58
685721    12/31/2011 23:58
685722    12/31/2011 23:58
685723    12/31/2011 23:58
Name: DATE_TIME, Length: 685724, dtype: object

## Formatting second DF

In [59]:
# making df16-df12 into one df

dfs_sixteen_to_eleven = [df_sixteen, df_fifteen, df_fourteen, 
                         df_thirteen, df_twelve, df_eleven]

df_2 = pd.concat(dfs_sixteen_to_eleven)

In [60]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1491240 entries, 0 to 685723
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype 
---  ------                       --------------    ----- 
 0   PRECINCT                     1491240 non-null  object
 1   SUSPECTED_CRIME_DESCRIPTION  1491231 non-null  object
 2   SUSPECT_ARRESTED_FLAG        1491240 non-null  object
 3   SUSPECT_RACE_DESCRIPTION     1491240 non-null  object
 4   DATE_TIME                    1491240 non-null  object
dtypes: object(5)
memory usage: 68.3+ MB


In [64]:
df_1.head()

,SUSPECTED_CRIME_DESCRIPTION,SUSPECT_ARRESTED_FLAG,SUSPECT_RACE_DESCRIPTION,PRECINCT
DATE_TIME,,,,
2021-01-01 01:50,CPW,Y,BLACK,42.0
2021-01-01 10:55,ROBBERY,Y,BLACK,105.0
2021-01-01 10:10,OTHER,Y,E. ASIAN,106.0
2021-01-01 20:17,ASSAULT,N,BLACK,73.0
2021-01-01 19:59,RAPE,N,BLACK,26.0


In [66]:
df_2.head()

,PRECINCT,SUSPECTED_CRIME_DESCRIPTION,SUSPECT_ARRESTED_FLAG,SUSPECT_RACE_DESCRIPTION,DATE_TIME
0,41,BURG,N,B,02/07/2016 01:00
1,10,MISDEMEANOR,N,W,02/18/2016 00:30
2,66,FEL,Y,P,01/01/2016 00:30
3,47,FEL,N,B,01/01/2016 00:40
4,79,D.W.I.,N,B,01/01/2016 00:50


In [67]:
df_2['DATE_TIME'] = pd.to_datetime(df_2['DATE_TIME'], format ='%m/%d/%Y %H:%M')
df_2['DATE_TIME'] = df_2['DATE_TIME'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M'))
df_2.set_index('DATE_TIME', drop = True, inplace = True)
df_2.head()

,PRECINCT,SUSPECTED_CRIME_DESCRIPTION,SUSPECT_ARRESTED_FLAG,SUSPECT_RACE_DESCRIPTION
DATE_TIME,,,,
2016-02-07 01:00,41,BURG,N,B
2016-02-18 00:30,10,MISDEMEANOR,N,W
2016-01-01 00:30,66,FEL,Y,P
2016-01-01 00:40,47,FEL,N,B
2016-01-01 00:50,79,D.W.I.,N,B


## Cleaning up race for second df

In [68]:
# overwritting our race map with values that are consistent with the other df

race_map = {'A': 'E. ASIAN', 'B': 'BLACK', 
 'I': 'NATIVE', 'P' : 'BLACK HISPANIC', 'Q': 'WHITE HISPANIC', 'W': 'WHITE', 
            'X' : 'UNKNOWN', 'Z' : 'OTHER'}

# mapping those races into new values for that column

df_2['SUSPECT_RACE_DESCRIPTION'] = df_2['SUSPECT_RACE_DESCRIPTION'].map(race_map)

In [69]:
df_1.head()
# inspecting the data

,SUSPECTED_CRIME_DESCRIPTION,SUSPECT_ARRESTED_FLAG,SUSPECT_RACE_DESCRIPTION,PRECINCT
DATE_TIME,,,,
2021-01-01 01:50,CPW,Y,BLACK,42.0
2021-01-01 10:55,ROBBERY,Y,BLACK,105.0
2021-01-01 10:10,OTHER,Y,E. ASIAN,106.0
2021-01-01 20:17,ASSAULT,N,BLACK,73.0
2021-01-01 19:59,RAPE,N,BLACK,26.0


# Making 1 DF

In [70]:
# putting the two dfs together 

df = pd.concat([df_1, df_2])
df.head()

# inspecting the data

,SUSPECTED_CRIME_DESCRIPTION,SUSPECT_ARRESTED_FLAG,SUSPECT_RACE_DESCRIPTION,PRECINCT
DATE_TIME,,,,
2021-01-01 01:50,CPW,Y,BLACK,42
2021-01-01 10:55,ROBBERY,Y,BLACK,105
2021-01-01 10:10,OTHER,Y,E. ASIAN,106
2021-01-01 20:17,ASSAULT,N,BLACK,73
2021-01-01 19:59,RAPE,N,BLACK,26


In [71]:
## setting precinct to 0 if precinct is nonsensical or unknown

In [72]:
df['PRECINCT'].isna().sum()

# looking for missing precicnt values 

431

In [73]:
df['PRECINCT'] = df['PRECINCT'].fillna(0)
df['PRECINCT'] = df['PRECINCT'].astype(str)

# setting empty precincts to 'precinct 0' so we will know what it means later
# setting the precinct to a string so we can get dummies

## filling in null vals for race description with "unknown"

In [74]:
df['SUSPECT_RACE_DESCRIPTION'] = df['SUSPECT_RACE_DESCRIPTION'].fillna('UNKNOWN')

# putting 'Unknown' for n/a race so we don't lose those rows

In [75]:
df['SUSPECT_RACE_DESCRIPTION'].value_counts()

# inspecting the data

BLACK             813776
WHITE HISPANIC    376358
WHITE             147096
BLACK HISPANIC    106227
E. ASIAN           53095
OTHER              30990
UNKNOWN            11816
NATIVE              6337
W. ASIAN             132
Name: SUSPECT_RACE_DESCRIPTION, dtype: int64

In [76]:
df = df.drop(columns="SUSPECTED_CRIME_DESCRIPTION")

## double checking all values

In [77]:
for x in df:
    print(df[x].value_counts().to_string())
    
# inspecting all values to make sure the make sense once more

N    1425853
Y     119974
BLACK             813776
WHITE HISPANIC    376358
WHITE             147096
BLACK HISPANIC    106227
E. ASIAN           53095
OTHER              30990
UNKNOWN            11816
NATIVE              6337
W. ASIAN             132
75          64226
73          56653
40          42758
44          37669
79          37542
120         36104
103         35669
23          33819
43          32021
83          31433
90          30807
67          30528
101         29644
70          28848
115         27884
32          27740
109         27617
52          26315
34          26057
81          25998
105         25594
77          25427
113         25310
114         24105
14          24062
42          23857
60          23672
47          23508
41          23208
110         23174
106         21466
46          21196
102         20596
25          20112
122         19755
49          17532
28          17474
30          17031
61          16622
88          15863
69          15422
72         

we can see:

   - we need to reformat strings with a .0 on the end of them
   - we need to get rid of the row where the precinct is 208760

In [78]:
# reformatting precinct information for consistancy 

df['PRECINCT'] = df['PRECINCT'].map(reformat_precinct)

In [79]:
for x in df:
    print(df[x].value_counts().to_string())
    
# inspecting the data

N    1425853
Y     119974
BLACK             813776
WHITE HISPANIC    376358
WHITE             147096
BLACK HISPANIC    106227
E. ASIAN           53095
OTHER              30990
UNKNOWN            11816
NATIVE              6337
W. ASIAN             132
75        66014
73        57793
40        44229
44        38752
79        38609
120       36724
103       36590
23        35229
43        33220
83        32114
67        31772
90        31429
101       30205
70        29413
32        28584
115       28567
109       28241
52        27564
34        26854
81        26704
77        26350
105       26268
113       25943
114       25217
42        25037
14        24875
60        24541
47        24287
110       23831
41        23633
46        22507
106       22138
25        21136
102       20928
122       20311
28        18211
49        18208
30        17514
61        17308
88        16325
69        16127
72        15642
71        15371
9         14557
104       14475
107       14409
33        139

In [80]:
df['PRECINCT'].replace({'208760': '0'}, inplace = True)

In [81]:
# making each stop and frisk have a unique ID and making a second df 
# where we can match that unique ID

In [82]:
df = df.sort_index()

In [83]:
df = df.reset_index(drop = False)
df = df.reset_index(drop = False)
df.head()

,index,DATE_TIME,SUSPECT_ARRESTED_FLAG,SUSPECT_RACE_DESCRIPTION,PRECINCT
0,0,2011-01-01 00:00,N,E. ASIAN,102
1,1,2011-01-01 00:10,N,BLACK,101
2,2,2011-01-01 00:10,N,BLACK,73
3,3,2011-01-01 00:10,N,BLACK,70
4,4,2011-01-01 00:15,N,WHITE HISPANIC,52


In [84]:
df = df.rename({'index' : 'Unique_ID'}, axis = 1)
df.head()

,Unique_ID,DATE_TIME,SUSPECT_ARRESTED_FLAG,SUSPECT_RACE_DESCRIPTION,PRECINCT
0,0,2011-01-01 00:00,N,E. ASIAN,102
1,1,2011-01-01 00:10,N,BLACK,101
2,2,2011-01-01 00:10,N,BLACK,73
3,3,2011-01-01 00:10,N,BLACK,70
4,4,2011-01-01 00:15,N,WHITE HISPANIC,52


In [85]:
df_race_blind = df.drop(columns = ['SUSPECT_RACE_DESCRIPTION'])
df_race_blind = df_race_blind.set_index("DATE_TIME")
df_race_blind

,Unique_ID,SUSPECT_ARRESTED_FLAG,PRECINCT
DATE_TIME,,,
2011-01-01 00:00,0,N,102
2011-01-01 00:10,1,N,101
2011-01-01 00:10,2,N,73
2011-01-01 00:10,3,N,70
2011-01-01 00:15,4,N,52
...,...,...,...
2021-12-31 23:03,1545822,N,104
2021-12-31 23:03,1545823,N,104
2021-12-31 23:03,1545824,N,104


In [86]:
df = df.set_index("DATE_TIME")
df

,Unique_ID,SUSPECT_ARRESTED_FLAG,SUSPECT_RACE_DESCRIPTION,PRECINCT
DATE_TIME,,,,
2011-01-01 00:00,0,N,E. ASIAN,102
2011-01-01 00:10,1,N,BLACK,101
2011-01-01 00:10,2,N,BLACK,73
2011-01-01 00:10,3,N,BLACK,70
2011-01-01 00:15,4,N,WHITE HISPANIC,52
...,...,...,...,...
2021-12-31 23:03,1545822,N,WHITE HISPANIC,104
2021-12-31 23:03,1545823,N,UNKNOWN,104
2021-12-31 23:03,1545824,N,WHITE HISPANIC,104


In [95]:
suspect_race_cat = df['SUSPECT_RACE_DESCRIPTION']
outcome = df['SUSPECT_ARRESTED_FLAG']

In [96]:
ten_years_tableau_data = pd.get_dummies(df, columns = ['SUSPECT_ARRESTED_FLAG',
                                                      'SUSPECT_RACE_DESCRIPTION'],)
ten_years_tableau_data["Counts"] = 1
ten_years_tableau_data["SUSPECT_RACE_CAT"] = suspect_race_cat
ten_years_tableau_data['SUSPECT_ARRESTED'] = outcome
# ten_years_tableau_data.to_csv("ten_years_tableau_data.csv")

In [97]:
ten_years_tableau_data

,Unique_ID,PRECINCT,SUSPECT_ARRESTED_FLAG_N,SUSPECT_ARRESTED_FLAG_Y,SUSPECT_RACE_DESCRIPTION_BLACK,SUSPECT_RACE_DESCRIPTION_BLACK HISPANIC,SUSPECT_RACE_DESCRIPTION_E. ASIAN,SUSPECT_RACE_DESCRIPTION_NATIVE,SUSPECT_RACE_DESCRIPTION_OTHER,SUSPECT_RACE_DESCRIPTION_UNKNOWN,SUSPECT_RACE_DESCRIPTION_W. ASIAN,SUSPECT_RACE_DESCRIPTION_WHITE,SUSPECT_RACE_DESCRIPTION_WHITE HISPANIC,Counts,SUSPECT_RACE_CAT,SUSPECT_ARRESTED
DATE_TIME,,,,,,,,,,,,,,,,
2011-01-01 00:00,0,102,1,0,0,0,1,0,0,0,0,0,0,1,E. ASIAN,N
2011-01-01 00:10,1,101,1,0,1,0,0,0,0,0,0,0,0,1,BLACK,N
2011-01-01 00:10,2,73,1,0,1,0,0,0,0,0,0,0,0,1,BLACK,N
2011-01-01 00:10,3,70,1,0,1,0,0,0,0,0,0,0,0,1,BLACK,N
2011-01-01 00:15,4,52,1,0,0,0,0,0,0,0,0,0,1,1,WHITE HISPANIC,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 23:03,1545822,104,1,0,0,0,0,0,0,0,0,0,1,1,WHITE HISPANIC,N
2021-12-31 23:03,1545823,104,1,0,0,0,0,0,0,1,0,0,0,1,UNKNOWN,N
2021-12-31 23:03,1545824,104,1,0,0,0,0,0,0,0,0,0,1,1,WHITE HISPANIC,N
